# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

05-10-2022


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2022-05-11 04:20:48,33.93911,67.709953,179017,7685,NaN,NaN,Afghanistan,459.862906,4.292888
1,NaN,NaN,NaN,Albania,2022-05-11 04:20:48,41.15330,20.168300,275416,3497,NaN,NaN,Albania,9570.366252,1.269716
2,NaN,NaN,NaN,Algeria,2022-05-11 04:20:48,28.03390,1.659600,265804,6875,NaN,NaN,Algeria,606.152059,2.586492
3,NaN,NaN,NaN,Andorra,2022-05-11 04:20:48,42.50630,1.521800,41717,153,NaN,NaN,Andorra,53992.105093,0.366757
4,NaN,NaN,NaN,Angola,2022-05-11 04:20:48,-11.20270,17.873900,99287,1900,NaN,NaN,Angola,302.093928,1.913644


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,178899,178901,178901,178901,178905,178919,178922,178981,179010,179017
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,275167,275177,275191,275211,275266,275310,275341,275366,275372,275416
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265782,265782,265782,265782,265786,265791,265794,265798,265800,265804


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7683,7683,7683,7683,7684,7684,7684,7684,7685,7685
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3496,3496,3496,3496,3496,3496,3496,3497,3497,3497
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6875,6875,6875,6875,6875,6875,6875,6875,6875,6875


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15827,15833,15835,15839,15840,15840,15840,15840,15846,15852
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,55643,55664,55685,55695,55713,55730,55730,55730,55761,55792
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,5668,5670,5671,5671,5671,5672,5672,5672,5675,5677


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,216,216,216,216,216,216,216,216,216,216
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,680,680,680,680,680,680,680,680,680,680
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,98,98,98,98,98,98,98,98,98,98


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
674,1001.0,Autauga,Alabama,US,2022-05-11 04:20:48,32.539527,-86.644082,15852,216,NaN,NaN,"Autauga, Alabama, US",28373.516619,1.362604
711,1075.0,Lamar,Alabama,US,2022-05-11 04:20:48,33.779950,-88.096680,3737,65,NaN,NaN,"Lamar, Alabama, US",27069.902209,1.739363
712,1077.0,Lauderdale,Alabama,US,2022-05-11 04:20:48,34.901719,-87.656247,24178,396,NaN,NaN,"Lauderdale, Alabama, US",26073.828037,1.637853


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,82059839,998048,0.0
India,43107689,524103,0.0
Brazil,30594388,664624,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,82059839,998048,0.0,81061791.0,2022-05-11 04:20:48,37.934269,-91.444083
India,43107689,524103,0.0,42583586.0,2022-05-11 04:20:48,23.088275,81.806127
Brazil,30594388,664624,0.0,29929764.0,2022-05-11 04:20:48,-12.669522,-48.480493
France,29212276,148016,0.0,29064260.0,2022-05-11 04:20:48,6.430808,-34.730285
Germany,25503878,136987,0.0,25366891.0,2022-05-11 04:20:48,51.571844,10.277881


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,9320578,90611,0.0
Texas,6854159,88191,0.0
Florida,6002153,74113,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,9320578,90611,0.0,9229967.0,2022-05-11 04:20:48,37.843962,-120.728594
Texas,6854159,88191,0.0,6765968.0,2022-05-11 04:20:48,31.660643,-98.653069
Florida,6002153,74113,0.0,5928040.0,2022-05-11 04:20:48,28.940755,-82.700744
New York,5254756,68529,0.0,5186227.0,2022-05-11 04:20:48,42.544151,-75.474183
Illinois,3189086,36117,0.0,3152969.0,2022-05-11 04:20:48,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,2897513,32000,0.0
Arizona,Maricopa,1279872,17305,0.0
Florida,Miami-Dade,1216226,10921,0.0
Illinois,Cook,1184612,14372,0.0
Texas,Harris,1029669,10966,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,2897513,32000,0.0,2865513.0,2022-05-11 04:20:48,34.308284,-118.228241,6037.0
Arizona,Maricopa,1279872,17305,0.0,1262567.0,2022-05-11 04:20:48,33.348359,-112.491815,4013.0
Florida,Miami-Dade,1216226,10921,0.0,1205305.0,2022-05-11 04:20:48,25.611236,-80.551706,12086.0
Illinois,Cook,1184612,14372,0.0,1170240.0,2022-05-11 04:20:48,41.841448,-87.816588,17031.0
Texas,Harris,1029669,10966,0.0,1018703.0,2022-05-11 04:20:48,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,178981,275366,265798,41717,99287,11,7663,9101319,422877,6286931,...,899723,238695,7785,522749,10676184,657060,535,11819,320099,248352
2022-05-09,179010,275372,265800,41717,99287,11,7663,9101319,422896,6334964,...,899723,238736,8056,522749,10678359,657193,535,11819,320157,248536
2022-05-10,179017,275416,265804,41717,99287,11,7663,9101319,422896,6392018,...,902540,238743,8063,522749,10681214,657193,535,11819,320226,248642


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,7684,3497,6875,153,1900,0,137,128729,8622,7516,...,7210,1637,14,5709,43056,5657,0,2149,3982,5476
2022-05-09,7685,3497,6875,153,1900,0,137,128729,8623,7559,...,7210,1637,14,5709,43057,5657,0,2149,3982,5479
2022-05-10,7685,3497,6875,153,1900,0,137,128729,8623,7616,...,7213,1637,14,5709,43058,5657,0,2149,3983,5481


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-05-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-05-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/1/22,5/2/22,5/3/22,5/4/22,5/5/22,5/6/22,5/7/22,5/8/22,5/9/22,5/10/22
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,15827,15833,15835,15839,15840,15840,15840,15840,15846,15852
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,3733,3733,3734,3734,3734,3735,3735,3735,3735,3737
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,24137,24141,24147,24151,24161,24167,24167,24167,24174,24178
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,7671,7672,7672,7673,7677,7678,7678,7678,7679,7680
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,37762,37776,37786,37789,37815,37818,37818,37818,37830,37849


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,1302397,254466,5923,2025435,837154,9288047,1389827,766172,264376,49,...,2031533,6848369,935479,126099,17167,1715478,1508065,502930,1623087,156745
2022-05-09,1302945,254466,5955,2025435,837216,9295307,1392887,770762,265472,49,...,2031533,6850868,935479,126937,17353,1721501,1519327,503807,1627977,156745
2022-05-10,1303441,254466,5969,2025435,837475,9320578,1394203,772709,265707,49,...,2031533,6854159,935479,127189,17485,1723985,1519327,503807,1630397,157031


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2022-05-08       15840   55730    5672  6444  14985    2318   5067   32421   
2022-05-09       15846   55761    5675  6447  14988    2318   5068   32431   
2022-05-10       15852   55792    5677  6450  14994    2318   5068   32442   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-05-08         8492     5127  ...    6787   1911     8067     1923   
2022-05-09         8495     5128  ...    6787   1911     8067     1923   
2022-05-10         8503     5130  ...    6801   1911     8097     1926   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2022-05-08          11075  10047  5635          0     2357   1589  
2022-05-09          11075  10047  5635          0     2357   1589  
2022-05-10          11088  10074  5643          0     2359   1588  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,19601,1253,28,30189,11408,90497,12487,10874,2928,0,...,26208,88176,4760,642,113,20280,12716,6880,14464,1814
2022-05-09,19601,1253,29,30189,11409,90503,12489,10875,2928,0,...,26208,88178,4760,644,113,20288,12742,6882,14475,1814
2022-05-10,19606,1253,29,30189,11409,90611,12497,10883,2928,0,...,26209,88191,4760,648,113,20295,12742,6882,14485,1817


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-05-08         216     680      98  104    243      54    129     626   
2022-05-09         216     680      98  104    243      54    129     626   
2022-05-10         216     680      98  104    243      54    129     626   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2022-05-08          162       86  ...     143     41       78       27   
2022-05-09          162       86  ...     143     41       78       27   
2022-05-10          162       86  ...     144     41       78       27   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-05-08            126    16    39          0       44     18  
2022-05-09            126    16    39          0       44     18  
2022-05-10            126    16    39          0       44     18  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,0.000330,0.000091,0.000015,0.0,0.0,0.0,0.0,0.001943,0.000000,0.006477,...,0.000000,0.000000,0.006594,0.000147,0.000213,0.000000,0.0,0.0,0.000000,0.000556
2022-05-09,0.000162,0.000022,0.000008,0.0,0.0,0.0,0.0,0.000000,0.000045,0.007640,...,0.000000,0.000172,0.034811,0.000000,0.000204,0.000202,0.0,0.0,0.000181,0.000741
2022-05-10,0.000039,0.000160,0.000015,0.0,0.0,0.0,0.0,0.000000,0.000000,0.009006,...,0.003131,0.000029,0.000869,0.000000,0.000267,0.000000,0.0,0.0,0.000216,0.000427


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,0.00000,0.000286,0.0,0.0,0.0,NaN,0.0,0.000591,0.000000,0.000932,...,0.000000,0.0,0.0,0.0,0.000023,0.0,NaN,0.0,0.000000,0.000548
2022-05-09,0.00013,0.000000,0.0,0.0,0.0,NaN,0.0,0.000000,0.000116,0.005721,...,0.000000,0.0,0.0,0.0,0.000023,0.0,NaN,0.0,0.000000,0.000548
2022-05-10,0.00000,0.000000,0.0,0.0,0.0,NaN,0.0,0.000000,0.000000,0.007541,...,0.000416,0.0,0.0,0.0,0.000023,0.0,NaN,0.0,0.000251,0.000365


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,0.000000,0.0,0.000000,0.0,0.000148,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000222,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-05-09,0.000421,0.0,0.005403,0.0,0.000074,0.000782,0.002202,0.005991,0.004146,0.0,...,0.0,0.000365,0.0,0.006646,0.010835,0.003511,0.007468,0.001744,0.003013,0.000000
2022-05-10,0.000381,0.0,0.002351,0.0,0.000309,0.002719,0.000945,0.002526,0.000885,0.0,...,0.0,0.000480,0.0,0.001985,0.007607,0.001443,0.000000,0.000000,0.001486,0.001825


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga   Baldwin   Barbour      Bibb  Blount Bullock   
2022-05-08      0.000000  0.000000  0.000000  0.000000  0.0000     0.0   
2022-05-09      0.000379  0.000556  0.000529  0.000465  0.0002     0.0   
2022-05-10      0.000379  0.000556  0.000352  0.000465  0.0004     0.0   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2022-05-08      0.000000  0.000000  0.000000  0.000000  ...  0.000000    0.0   
2022-05-09      0.000197  0.000308  0.000353  0.000195  ...  0.000000    0.0   
2022-05-10      0.000000  0.000339  0.000942  0.000390  ...  0.002063    0.0   

Province_State                                                              \
Admin2          Sheridan Sublette Sweetwater     Teton    Uinta Unassigned   
2022-05-08      0.000000  0.00000   0.000000  0.000000  0.00000        NaN   
2022-05-09      0.000000  0.00000   0.000000  0.000000  0.00000        NaN   
2022-05-10      0.003719  0.00156   0.001174  0.002687  0.00142        NaN   

Province_State                      
Admin2          Washakie    Weston  
2022-05-08      0.000000  0.000000  
2022-05-09      0.000000  0.000000  
2022-05-10      0.000848 -0.000629  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,0.000000,0.0,0.000000,0.0,0.000088,0.000000,0.000000,0.000000,0.0,NaN,...,0.000000,0.000011,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2022-05-09,0.000000,0.0,0.035714,0.0,0.000088,0.000066,0.000160,0.000092,0.0,NaN,...,0.000000,0.000023,0.0,0.003115,0.0,0.000394,0.002045,0.000291,0.000760,0.000000
2022-05-10,0.000255,0.0,0.000000,0.0,0.000000,0.001193,0.000641,0.000736,0.0,NaN,...,0.000038,0.000147,0.0,0.006211,0.0,0.000345,0.000000,0.000000,0.000691,0.001654


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2022-05-08         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-05-09         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2022-05-10         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ...   Wyoming                           \
Admin2         Chambers Cherokee  ...      Park Platte Sheridan Sublette   
2022-05-08          0.0      0.0  ...  0.000000    0.0      0.0      0.0   
2022-05-09          0.0      0.0  ...  0.000000    0.0      0.0      0.0   
2022-05-10          0.0      0.0  ...  0.006993    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2022-05-08            0.0   0.0   0.0        NaN      0.0    0.0  
2022-05-09            0.0   0.0   0.0        NaN      0.0    0.0  
2022-05-10            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,0.000181,0.000111,0.000014,0.000282,0.0,0.0,0.000193,0.000976,4.000000e-07,0.006454,...,0.000035,0.000048,0.005045,0.000114,0.000274,0.000000,0.0,7.000000e-07,0.000271,0.000374
2022-05-09,0.000172,0.000066,0.000011,0.000141,0.0,0.0,0.000097,0.000488,2.260000e-05,0.007047,...,0.000017,0.000110,0.019928,0.000057,0.000239,0.000101,0.0,3.000000e-07,0.000226,0.000557
2022-05-10,0.000105,0.000113,0.000013,0.000071,0.0,0.0,0.000048,0.000244,1.130000e-05,0.008027,...,0.001574,0.000069,0.010398,0.000029,0.000253,0.000051,0.0,2.000000e-07,0.000221,0.000492


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,0.000008,0.000143,0.0,-0.0,0.0,NaN,0.000233,0.000299,0.000000,0.002359,...,0.000016,0.0,0.002414,7.000000e-07,0.000035,0.0,NaN,0.0,0.000377,0.000327
2022-05-09,0.000069,0.000072,0.0,-0.0,0.0,NaN,0.000117,0.000149,0.000058,0.004040,...,0.000008,0.0,0.001207,3.000000e-07,0.000029,0.0,NaN,0.0,0.000189,0.000437
2022-05-10,0.000035,0.000036,0.0,-0.0,0.0,NaN,0.000058,0.000075,0.000029,0.005790,...,0.000212,0.0,0.000604,2.000000e-07,0.000026,0.0,NaN,0.0,0.000220,0.000401


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2022-05-08,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-05-09,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2022-05-10,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,0.000055,0.000182,0.001173,0.000061,0.000198,0.000264,0.000218,0.000481,0.000364,0.0,...,0.000049,0.000337,0.000217,0.000791,0.002519,0.000381,0.000022,0.000167,0.000379,0.000020
2022-05-09,0.000238,0.000091,0.003288,0.000030,0.000136,0.000523,0.001210,0.003236,0.002255,0.0,...,0.000024,0.000351,0.000109,0.003718,0.006677,0.001946,0.003745,0.000956,0.001696,0.000010
2022-05-10,0.000309,0.000046,0.002819,0.000015,0.000223,0.001621,0.001077,0.002881,0.001570,0.0,...,0.000012,0.000416,0.000054,0.002852,0.007142,0.001694,0.001872,0.000478,0.001591,0.000917


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2022-05-08      0.000017  0.000073  0.000028  0.000015  0.000014   
2022-05-09      0.000198  0.000315  0.000279  0.000240  0.000107   
2022-05-10      0.000288  0.000435  0.000316  0.000353  0.000254   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2022-05-08      1.000000e-07  0.000009  0.000018  0.000021  0.000026  ...   
2022-05-09      0.000000e+00  0.000103  0.000163  0.000187  0.000111  ...   
2022-05-10      0.000000e+00  0.000052  0.000251  0.000565  0.000250  ...   

Province_State   Wyoming                                                     \
Admin2              Park    Platte  Sheridan  Sublette Sweetwater     Teton   
2022-05-08      0.000023  0.000016  0.000019  0.000008   0.000008  0.000060   
2022-05-09      0.000012  0.000008  0.000010  0.000004   0.000004  0.000030   
2022-05-10      0.001037  0.000004  0.001864  0.000782   0.000589  0.001359   

Province_State                                           
Admin2             Uinta Unassigned  Washakie    Weston  
2022-05-08      0.000022       -1.0 -0.000007  0.000010  
2022-05-09      0.000011       -1.0 -0.000003  0.000005  
2022-05-10      0.000715       -1.0  0.000423 -0.000312  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2022-05-08,0.000082,0.000026,0.005876,0.000249,0.000143,0.000145,0.000555,0.000200,0.000418,NaN,...,0.000098,0.000082,0.000172,0.000326,0.000563,0.000128,0.000028,0.000123,0.000210,0.000018
2022-05-09,0.000041,0.000013,0.020795,0.000125,0.000116,0.000106,0.000358,0.000146,0.000209,NaN,...,0.000049,0.000052,0.000086,0.001721,0.000282,0.000262,0.001037,0.000207,0.000485,0.000009
2022-05-10,0.000148,0.000006,0.010398,0.000062,0.000058,0.000650,0.000499,0.000441,0.000105,NaN,...,0.000044,0.000100,0.000043,0.003966,0.000141,0.000303,0.000518,0.000103,0.000588,0.000831


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                               \
Admin2               Autauga Baldwin Barbour          Bibb Blount Bullock   
2022-05-08      1.100000e-06     0.0     0.0  1.000000e-07    0.0     0.0   
2022-05-09      6.000000e-07     0.0     0.0  0.000000e+00    0.0     0.0   
2022-05-10      3.000000e-07     0.0     0.0  0.000000e+00    0.0     0.0   

Province_State                                   ...   Wyoming         \
Admin2         Butler Calhoun Chambers Cherokee  ...      Park Platte   
2022-05-08        0.0     0.0      0.0      0.0  ...  0.000000    0.0   
2022-05-09        0.0     0.0      0.0      0.0  ...  0.000000    0.0   
2022-05-10        0.0     0.0      0.0      0.0  ...  0.003497    0.0   

Province_State                                                               \
Admin2         Sheridan Sublette Sweetwater Teton Uinta Unassigned Washakie   
2022-05-08          0.0      0.0   0.000126   0.0   0.0       -1.0      0.0   
2022-05-09          0.0      0.0   0.000063   0.0   0.0       -1.0      0.0   
2022-05-10          0.0      0.0   0.000031   0.0   0.0       -1.0      0.0   

Province_State         
Admin2         Weston  
2022-05-08        0.0  
2022-05-09        0.0  
2022-05-10        0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [89]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [90]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [91]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [92]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


#### 